<a href="https://colab.research.google.com/github/AravindReddy123/Aravind_INFO5731_Spring2023/blob/main/INFO5731_Assignment_Three/INFO5731_Assignment_Three_fall2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:
# Write your code here

import requests
import json
import pandas as pd
import time
from google.colab import  drive
drive.mount('/drive', force_remount=True)
#Initializing the limit to 100 so that for everytime we hit the API we get only 100 records.
limit="100"
#Declaring the URL
url = "https://api.semanticscholar.org/graph/v1/paper/search?query=machine%20learning+data%20science+artificial%20intelligence&fields=abstract&offset="
appended_data = []
for i in range(0,1000,100):
  offset = str(i)
  #Creating the dynamic url with offset changing for every iteration of loop
  dynamic_url = url+offset+"&limit="+limit
  time.sleep(5)
  #Calling get method from request library and sending url as parameter in get method to get the required data from API.
  response = requests.get(dynamic_url,headers={"x-api-key":"wMeN1WdWq75wh1hHr5CM34dDRJCaS5MQFD6s67kb"})
  #Converting the text response we got from the above to JSON format
  print(response.text)
  data_response = json.loads(response.text)['data']
  #Converting the JSON data format to pandas Dataframe.
  df = pd.DataFrame(data_response)
  #df = df.dropna(axis=0,subset = ['abstract'])
  #concatenating the dataframes for each offset to appended_data variable
  appended_data.append(df)
#print(appended_data[0])
appended_data = pd.concat(appended_data,ignore_index=True)
appended_data = pd.DataFrame(appended_data,columns=['paperId','abstract'])
appended_data = appended_data.dropna(axis=0,subset = ['abstract'])

Mounted at /drive
{"total": 101524, "offset": 0, "next": 100, "data": [{"paperId": "1e85606f75af68884122f0d1d8393c000e915194", "abstract": "This Special Issue of the IEEE Transactions on Plasma Science (TPS) follows the first American Physical Society Division of Plasma Physics (APS-DPP) mini-conference on Machine Learning, Data Science, and Artificial Intelligence in Plasma Research held during the 60th APS-DPP Meeting in Portland, OR, USA (November 5\u20139, 2018). It contains selected highlights from not only the mini-conference but also the broader plasma physics community. Although data science has a long and rich history in plasma physics, dating back at least three decades, it is experiencing a renaissance, thanks in large part to the advances outside of plasma physics. Novel algorithms, hardware, and analytic techniques (buoyed by the open source software ecosystem) have led plasma scientists to explore ways in which the data revolution could accelerate and inform scientific di

In [3]:
appended_data

,paperId,abstract,sentiment
0,1e85606f75af68884122f0d1d8393c000e915194,This Special Issue of the IEEE Transactions on...,positive
2,72d3ddf1f7210d7e70144bbc09f770ec411fe909,Smarter applications are making better use of ...,positive
3,a20c39f5439b84b57d883a21b698394d2347800d,"Over the years, many clinical and engineering ...",negative
4,ddb9ba8651e39d3c2e7a990c4b27049c20aff228,Machine learning (ML) is becoming capable of t...,positive
9,12d89245440d8c2a57c0741d10177189adf230d3,Abstract The Petaflops supercomputer “Zhores” ...,positive
...,...,...,...
995,ab95f9e5697a7619da9b4a8dbf81f338f2b2ec21,We consider the use of intelligent systems to ...,positive
996,c59f6b9b3644bd776433828aa88c6c0d00f42622,Expert systems are part of a general category ...,positive
997,9fd6621c52fbfff2778448515a83df000358597e,"In 1989, B. F. Skinner told Joseph Rychlak tha...",positive
998,55d0b6681645c91a4436a36d9aec4a1b3f542e97,Carbon nanotubes have an exciting array of app...,positive


In [18]:
#This code generate ngrams like bi-grams,tri-grams for the given data
def generate_ngrams(data, n):
    ngrams = []
    words = data.split()
    for i in range(len(words)-(n-1)):
        ngram = ' '.join(words[i:i+n])
        ngrams.append(ngram)
    return ngrams

In [19]:
def count_ngrams(ngrams):
    ngram_frequency = {}
    for ngram in ngrams:
        if ngram in ngram_frequency:
            ngram_frequency[ngram] += 1
        else:
            ngram_frequency[ngram] = 1
    return ngram_frequency

In [20]:
#Pre-Processing and cleaning the data
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop = stopwords.words('english')
#ppended_data[['abstract','stopwords']].head()
appended_data['abstract'] = appended_data['abstract'].str.replace('[^\w\s]','')
appended_data.head()
appended_data['abstract'] = appended_data['abstract'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
appended_data['abstract'].head()
appended_data['stopwords'] = appended_data['abstract'].apply(lambda x: len([x for x in x.split() if x in stop]))
appended_data['abstract'] = appended_data['abstract'].apply(lambda x: " ".join(x.lower() for x in x.split()))
st = PorterStemmer()
appended_data['abstract'] = appended_data['abstract'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
appended_data['abstract'] = appended_data['abstract'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
appended_data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
<ipython-input-20-863f0ab218cc>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  appended_data['abstract'] = appended_data['abstract'].str.replace('[^\w\s]','')


,paperId,abstract,stopwords
0,1e85606f75af68884122f0d1d8393c000e915194,thi special issu ieee transact plasma scienc t...,0
2,72d3ddf1f7210d7e70144bbc09f770ec411fe909,smarter applic make better use insight glean d...,0
3,a20c39f5439b84b57d883a21b698394d2347800d,year mani clinic engin method adapt test scree...,0
4,ddb9ba8651e39d3c2e7a990c4b27049c20aff228,machin learn ml becom capabl transform biomole...,0
9,12d89245440d8c2a57c0741d10177189adf230d3,abstract petaflop supercomput zhore recent lau...,0


In [21]:
abstracts = ' '.join(appended_data['abstract'].tolist())

In [22]:
#Count the frequency of all the N-grams (N=3)
n = 3
ngrams = generate_ngrams(abstracts, n)
ngram_frequency = count_ngrams(ngrams)
print("Frequency of", n, "-grams:")
for ngram, frequency in ngram_frequency.items():
    print(ngram, ":", frequency)

Streaming output truncated to the last 5000 lines.
import compon data : 2
data scienc botnet : 1
scienc botnet visual : 1
botnet visual microsoft : 1
visual microsoft digit : 1
microsoft digit crime : 1
digit crime unit : 1
crime unit tap : 1
unit tap offic : 1
tap offic creativ : 1
offic creativ research : 1
creativ research multidisciplinari : 1
research multidisciplinari digit : 1
multidisciplinari digit design : 1
digit design group : 1
design group base : 1
group base new : 1
base new york : 1
new york come : 1
york come new : 1
come new way : 1
new way look : 1
way look one : 1
look one particular : 1
one particular threat : 1
particular threat botnet : 1
threat botnet global : 1
botnet global network : 1
global network infect : 1
network infect comput : 1
infect comput cyber : 1
comput cyber crimin : 1
cyber crimin enlist : 1
crimin enlist bid : 1
enlist bid ocr : 1
bid ocr came : 1
ocr came prototyp : 1
came prototyp tool : 1
prototyp tool call : 1
tool call specimen : 1
call s

In [7]:
#Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.
def generate_bigrams(text):
    words = text.split()
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    return bigrams


def calculate_bigram_probabilities(bigrams):
    bigram_frequency = {}
    unigram_frequency = {}
    
    # Count the frequencies of bigrams and unigrams
    for bigram in bigrams:
        if bigram in bigram_frequency:
            bigram_frequency[bigram] += 1
        else:
            bigram_frequency[bigram] = 1
        
        if bigram[0] in unigram_frequency:
            unigram_frequency[bigram[0]] += 1
        else:
            unigram_frequency[bigram[0]] = 1
    
     # Calculate the probabilities for bigrams
    bigram_prob = {}
    for bigram in bigram_frequency:
        w2_w1_count = bigram_frequency[bigram]
        w2_count = unigram_frequency[bigram[0]]
        bigram_prob[bigram] = w2_w1_count / w2_count
    
    return bigram_prob

In [8]:
bigrams = generate_bigrams(abstracts)
bigram_probability = calculate_bigram_probabilities(bigrams)

# Displaying the results
print("Probabilities for all the bigrams:")
for bigram, prob in bigram_probability.items():
    print(bigram[0], bigram[1], ":", prob)

Streaming output truncated to the last 5000 lines.
translat particularli : 0.018518518518518517
particularli materi : 0.017543859649122806
materi doublesum : 0.005405405405405406
doublesum methodthi : 0.5
methodthi monograph : 1.0
monograph open : 0.5
horizon engin : 0.1111111111111111
research academia : 0.0010857763300760044
industri deal : 0.0047169811320754715
deal interest : 0.02127659574468085
interest new : 0.007518796992481203
identif control : 0.021739130434782608
control it : 0.007936507936507936
it emphas : 0.004048582995951417
emphas guidelin : 0.038461538461538464
guidelin work : 0.047619047619047616
work solut : 0.0038910505836575876
practic advic : 0.004784688995215311
advic implement : 0.25
implement rather : 0.005917159763313609
rather theoret : 0.02040816326530612
background gaussian : 0.02
process gp : 0.0017793594306049821
gp model : 0.42857142857142855
book demonstr : 0.009615384615384616
probabilist machinelearn : 0.030303030303030304
give reader : 0.0227272727272

In [13]:
# (3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews 
# (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. 
# Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 

import pandas as pd
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_noun_phrases(text):
    noun_phrases = []
    pos_tags = pos_tag(word_tokenize(text))
    current_noun_phrase = []
    for word, pos in pos_tags:
      # Check if the word has a noun part of speech tag
        if pos.startswith('NN'):
            current_noun_phrase.append(word)
        else:
            if current_noun_phrase:
                noun_phrases.append(' '.join(current_noun_phrase))
                current_noun_phrase = []
    if current_noun_phrase:
        noun_phrases.append(' '.join(current_noun_phrase))
    return noun_phrases

def calculate_noun_phrase_probabilities(noun_phrases):
    noun_phrase_frequency = {}
    for noun_phrase in noun_phrases:
        if noun_phrase in noun_phrase_frequency:
            noun_phrase_frequency[noun_phrase] += 1
        else:
            noun_phrase_frequency[noun_phrase] = 1
    max_freq = max(noun_phrase_frequency.values()) if noun_phrase_frequency else 1  # Check if the list is empty
    noun_phrase_prob = {}
    for noun_phrase in noun_phrase_frequency:
        noun_phrase_prob[noun_phrase] = noun_phrase_frequency[noun_phrase] / max_freq
    
    return noun_phrase_prob


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [14]:
review_noun_phrases = {}
for i, text in enumerate(appended_data['abstract']):
    noun_phrases = extract_noun_phrases(text)
    noun_phrase_probability = calculate_noun_phrase_probabilities(noun_phrases)
    review_noun_phrases[f'Review {i+1}'] = noun_phrase_probability

noun_phrases_results = pd.DataFrame(review_noun_phrases)
noun_phrases_results.fillna(0, inplace=True)
noun_phrases_results.index.name = 'Review'
noun_phrases_results.columns.name = 'Noun Phrase'
print(noun_phrases_results)

Relative Probabilities of Noun Phrases for Each Review:
Noun Phrase                                         Review 1  Review 2  \
Review                                                                   
thi                                                      0.5       0.0   
issu ieee transact plasma scienc tp                      0.5       0.0   
physic societi divis plasma                              0.5       0.0   
apsdpp miniconfer machin learn data scienc arti...       0.5       0.0   
research                                                 1.0       0.0   
...                                                      ...       ...   
lowlevel data gather                                     0.0       0.0   
sensor camera wearabl sensor instrument user             0.0       0.0   
primari task space discov                                0.0       0.0   
data interpret                                           0.0       0.0   
activ recognit system process                           

# **Question 2: Undersand TF-IDF and Document representation**

(20 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [15]:
# Write your code here

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(appended_data['abstract'])
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Printing the tf-idf matrix
print(df_tfidf)


Documents-Terms Weights (tf-idf) Matrix:
     000  0000  0001  0005  001  00489  005  006  00905   01  ...   κ2   π2  \
0    0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
1    0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
2    0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
3    0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
4    0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
..   ...   ...   ...   ...  ...    ...  ...  ...    ...  ...  ...  ...  ...   
765  0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
766  0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
767  0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
768  0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   
769  0.0   0.0   0.0   0.0  0.0    0.0  0.0  0.0    0.0  0.0  ...  0.0  0.0   

      σk  

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(appended_data['abstract'])

query = "Novel algorithms, hardware, and analytic techniques (buoyed by the open source software ecosystem) have led plasma scientists to explore ways in which the data revolution could accelerate and inform scientific discovery"
query_vector = vectorizer.transform([query])

cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)

similarity_scores = cosine_similarities.flatten()

similarity_dataframe = pd.DataFrame({'Document': appended_data.index, 'Similarity': similarity_scores})

similarity_dataframe = similarity_dataframe.sort_values(by='Similarity', ascending=False)
print(similarity_dataframe)

     Document  Similarity
0           0    0.554790
107       138    0.217563
622       804    0.156310
290       376    0.113480
4           9    0.108753
..        ...         ...
37         51    0.000000
146       183    0.000000
307       399    0.000000
391       501    0.000000
667       862    0.000000

[770 rows x 2 columns]


# **Question 3: Create your own word embedding model**

(20 points). Use the data you collected for assignment two to build a word embedding model: 

(1) Train a 300-dimension word embedding (it can be word2vec, glove, ulmfit, bert, or others).

(2) Visualize the word embedding model you created.

Reference: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

Reference: https://jaketae.github.io/study/word2vec/

In [23]:
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

model = Word2Vec.load('word2vec_model.bin')
vocabulary = list(model.wv.key_to_index.keys())
embeddings = model.wv[vocabulary]

tsne = TSNE(n_components=2, random_state=42)
embeddings_2dim = tsne.fit_transform(embeddings)
plt.figure(figsize=(10, 8))
plt.scatter(embeddings_2dim[:, 0], embeddings_2dim[:, 1], marker='o')
for i, word in enumerate(vocabulary):
    plt.annotate(word, (embeddings_2dim[i, 0], embeddings_2dim[i, 1]))
plt.xlabel('t-SNE Dim 1')
plt.ylabel('t-SNE Dim 2')
plt.title('Word Embeddings Visualization')
plt.show()

# **Question 4: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [6]:
# The GitHub link of your final csv file
https://github.com/AravindReddy123/Aravind_INFO5731_Spring2023/blob/main/abstract_sentiments.xlsx